In [ ]:
pip install keras-tuner

# try

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torch.utils.data import Dataset, DataLoader, random_split


In [77]:

video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv').drop(columns=['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness'])
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv').drop(columns=['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness'])

scaler_video = StandardScaler()
video_features = scaler_video.fit_transform(video_features_df.iloc[:, 1:].values)

scaler_audio = StandardScaler()
audio_features = scaler_audio.fit_transform(audio_features_df.iloc[:, 1:].values)


In [78]:
assert len(video_features) == len(audio_features)

In [79]:
class VideoAudioDataset(Dataset):
    def __init__(self, video_features, audio_features):
        self.video_features = video_features
        self.audio_features = audio_features
        
    def __len__(self):
        return len(self.video_features)
    
    def __getitem__(self, idx):
        video = self.video_features[idx]
        audio = self.audio_features[idx]
        positive_pair = (video, audio)
        
        random_idx = random.choice([i for i in range(len(self.video_features)) if i != idx])
        negative_audio = self.audio_features[random_idx]
        negative_pair = (video, negative_audio)
        
        return positive_pair, negative_pair

dataset = VideoAudioDataset(video_features, audio_features)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

class ComplexCNNEncoder(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.5):
        super(ComplexCNNEncoder, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=hidden_sizes[0], kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])
        self.dropout1 = nn.Dropout(dropout_rate)
        self.conv2 = nn.Conv1d(in_channels=hidden_sizes[0], out_channels=hidden_sizes[1], kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])
        self.dropout2 = nn.Dropout(dropout_rate)
        self.conv3 = nn.Conv1d(in_channels=hidden_sizes[1], out_channels=hidden_sizes[2], kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])
        self.fc = nn.Linear(hidden_sizes[2], output_size)
        
    def forward(self, x):
        x = x.unsqueeze(1)  
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.avg_pool1d(x, kernel_size=x.size()[2]) 
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

video_encoder = ComplexCNNEncoder(input_size=1, hidden_sizes=[128, 64, 32], output_size=128, dropout_rate=0.5)
audio_encoder = ComplexCNNEncoder(input_size=1, hidden_sizes=[128, 64, 32], output_size=128, dropout_rate=0.5)


In [ ]:
def contrastive_loss(z_i, z_j, temperature=0.1):
    cos_sim = F.cosine_similarity(z_i, z_j) / temperature
    loss = -torch.log(torch.exp(cos_sim) / torch.sum(torch.exp(cos_sim)))
    return torch.mean(loss)

In [ ]:
total_dataset = VideoAudioDataset(video_features, audio_features)
train_size = int(0.8 * len(total_dataset))
val_size = len(total_dataset) - train_size
train_dataset, val_dataset = random_split(total_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
video_encoder = ComplexCNNEncoder(input_size=1, hidden_sizes=[128, 64, 32], output_size=128, dropout_rate=0.5)
audio_encoder = ComplexCNNEncoder(input_size=1, hidden_sizes=[128, 64, 32], output_size=128, dropout_rate=0.5)

In [85]:
def train_epoch(models, dataloader, optimizer):
    video_encoder, audio_encoder = models
    video_encoder.train()
    audio_encoder.train()
    total_loss = 0
    for positive_pair, negative_pair in dataloader:
        optimizer.zero_grad()
        
        video_positive, audio_positive = positive_pair[0].to(torch.float32), positive_pair[1].to(torch.float32)
        video_negative, audio_negative = negative_pair[0].to(torch.float32), negative_pair[1].to(torch.float32)

        z_i_positive = video_encoder(video_positive)
        z_j_positive = audio_encoder(audio_positive)
        z_i_negative = video_encoder(video_negative)
        z_j_negative = audio_encoder(audio_negative)
        
        loss_positive = contrastive_loss(z_i_positive, z_j_positive)
        loss_negative = contrastive_loss(z_i_negative, z_j_negative)
        
        loss = loss_positive + loss_negative
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_epoch(models, dataloader):
    video_encoder, audio_encoder = models
    video_encoder.eval()
    audio_encoder.eval()
    total_loss = 0
    with torch.no_grad():
        for positive_pair, negative_pair in dataloader:
            video_positive, audio_positive = positive_pair[0].to(torch.float32), positive_pair[1].to(torch.float32)
            video_negative, audio_negative = negative_pair[0].to(torch.float32), negative_pair[1].to(torch.float32)

            z_i_positive = video_encoder(video_positive)
            z_j_positive = audio_encoder(audio_positive)
            z_i_negative = video_encoder(video_negative)
            z_j_negative = audio_encoder(audio_negative)
            
            loss_positive = contrastive_loss(z_i_positive, z_j_positive)
            loss_negative = contrastive_loss(z_i_negative, z_j_negative)
            
            loss = loss_positive + loss_negative
            
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [ ]:
optimizer = torch.optim.Adam(list(video_encoder.parameters()) + list(audio_encoder.parameters()), lr=0.0001)

# Training 
num_epochs = 100
for epoch in range(num_epochs):
    train_loss = train_epoch((video_encoder, audio_encoder), train_dataloader, optimizer)
    val_loss = evaluate_epoch((video_encoder, audio_encoder), val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

Epoch 1/100, Train Loss: 6.8025, Val Loss: 4.1591
Epoch 2/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 3/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 4/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 5/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 6/100, Train Loss: 6.8025, Val Loss: 4.1591
Epoch 7/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 8/100, Train Loss: 6.8025, Val Loss: 4.1591
Epoch 9/100, Train Loss: 6.8025, Val Loss: 4.1594
Epoch 10/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 11/100, Train Loss: 6.8025, Val Loss: 4.1592
Epoch 12/100, Train Loss: 6.8025, Val Loss: 4.1590
Epoch 13/100, Train Loss: 6.8025, Val Loss: 4.1590
Epoch 14/100, Train Loss: 6.8025, Val Loss: 4.1590
Epoch 15/100, Train Loss: 6.8025, Val Loss: 4.1590
Epoch 16/100, Train Loss: 6.8025, Val Loss: 4.1590
Epoch 17/100, Train Loss: 6.8025, Val Loss: 4.1590
Epoch 18/100, Train Loss: 6.8025, Val Loss: 4.1591
Epoch 19/100, Train Loss: 6.8025, Val Loss: 4.1591
Epoch 20/100, Train Loss: 6.8025, Val Lo

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    train_loss = train((video_encoder, audio_encoder), train_dataloader, optimizer)
    val_loss = evaluate((video_encoder, audio_encoder), val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

C:\Users\giris\AppData\Local\Temp\ipykernel_60224\4194671997.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  video_positive = torch.tensor(video_positive, dtype=torch.float32)
C:\Users\giris\AppData\Local\Temp\ipykernel_60224\4194671997.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio_positive = torch.tensor(audio_positive, dtype=torch.float32)
C:\Users\giris\AppData\Local\Temp\ipykernel_60224\4194671997.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  video_negative = torch.tensor(video_negative, dtype=torch.float32)
C

Epoch 1/100, Train Loss: 6.8055, Val Loss: 4.1604
Epoch 2/100, Train Loss: 6.8036, Val Loss: 4.1594
Epoch 3/100, Train Loss: 6.8028, Val Loss: 4.1596
Epoch 4/100, Train Loss: 6.8029, Val Loss: 4.1607
Epoch 5/100, Train Loss: 6.8034, Val Loss: 4.1624
Epoch 6/100, Train Loss: 6.8040, Val Loss: 4.1618
Epoch 7/100, Train Loss: 6.8049, Val Loss: 4.1626
Epoch 8/100, Train Loss: 6.8050, Val Loss: 4.1615
Epoch 9/100, Train Loss: 6.8056, Val Loss: 4.1624
Epoch 10/100, Train Loss: 6.8046, Val Loss: 4.1615
Epoch 11/100, Train Loss: 6.8040, Val Loss: 4.1604
Epoch 12/100, Train Loss: 6.8036, Val Loss: 4.1594
Epoch 13/100, Train Loss: 6.8029, Val Loss: 4.1593
Epoch 14/100, Train Loss: 6.8026, Val Loss: 4.1590
Epoch 15/100, Train Loss: 6.8026, Val Loss: 4.1590
Epoch 16/100, Train Loss: 6.8027, Val Loss: 4.1593
Epoch 17/100, Train Loss: 6.8030, Val Loss: 4.1598
Epoch 18/100, Train Loss: 6.8032, Val Loss: 4.1599
Epoch 19/100, Train Loss: 6.8035, Val Loss: 4.1597
Epoch 20/100, Train Loss: 6.8032, Val Lo

#  

In [ ]:
def create_encoder(input_shape, name):
    input_layer = keras.Input(shape=input_shape)
    x = layers.Conv1D(32, 3, activation='relu', padding='same')(input_layer)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    return keras.Model(inputs=input_layer, outputs=x, name=name)

class DualEncoder(keras.Model):
    def __init__(self, encoder_a, encoder_v, **kwargs):
        super().__init__(**kwargs)
        self.encoder_a = encoder_a
        self.encoder_v = encoder_v
        self.regression_head = keras.layers.Dense(5, activation=None)  # For predicting personality traits
        
    def call(self, inputs):
        audio_input, video_input = inputs
        audio_embedding = self.encoder_a(audio_input)
        video_embedding = self.encoder_v(video_input)
        combined_embedding = keras.layers.concatenate([audio_embedding, video_embedding], axis=-1)
        personality_scores_pred = self.regression_head(combined_embedding)
        return audio_embedding, video_embedding, personality_scores_pred

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            audio_embedding, video_embedding, personality_scores_pred = self(x, training=True)
            loss = self.compute_loss(y, personality_scores_pred, audio_embedding, video_embedding)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss}

    def compute_loss(self, true_scores, pred_scores, audio_embedding, video_embedding):
        mse_loss = tf.reduce_mean(keras.losses.mean_squared_error(true_scores, pred_scores))
        cosine_loss = tf.reduce_mean(1 - tf.keras.losses.cosine_similarity(audio_embedding, video_embedding))
        return mse_loss + cosine_loss

encoder_a = create_encoder(input_shape=(audio_features.shape[1], 1), name="audio_encoder")
encoder_v = create_encoder(input_shape=(video_features.shape[1], 1), name="video_encoder")

dual_encoder = DualEncoder(encoder_a, encoder_v)
dual_encoder.compile(optimizer=keras.optimizers.Adam())

batch_size = 32

audio_features_expanded = np.expand_dims(audio_features, axis=-1)
video_features_expanded = np.expand_dims(video_features, axis=-1)

dataset = tf.data.Dataset.from_tensor_slices(((audio_features_expanded, video_features_expanded), personality_scores))
dataset = dataset.batch(batch_size)

num_epochs = 200
dual_encoder.fit(dataset, epochs=num_epochs)

Epoch 1/200
2/2 [==============================] - 1s 15ms/step - loss: 2.1153
Epoch 2/200
2/2 [==============================] - 0s 15ms/step - loss: 1.4037
Epoch 3/200
2/2 [==============================] - 0s 15ms/step - loss: 1.2035
Epoch 4/200
2/2 [==============================] - 0s 16ms/step - loss: 1.1896
Epoch 5/200
2/2 [==============================] - 0s 12ms/step - loss: 1.1658
Epoch 6/200
2/2 [==============================] - 0s 12ms/step - loss: 1.1113
Epoch 7/200
2/2 [==============================] - 0s 16ms/step - loss: 1.0601
Epoch 8/200
2/2 [==============================] - 0s 12ms/step - loss: 1.0332
Epoch 9/200
2/2 [==============================] - 0s 14ms/step - loss: 1.0309
Epoch 10/200
2/2 [==============================] - 0s 11ms/step - loss: 1.0377
Epoch 11/200
2/2 [==============================] - 0s 12ms/step - loss: 1.0398
Epoch 12/200
2/2 [==============================] - 0s 13ms/step - loss: 1.0343
Epoch 13/200
2/2 [==============================]

In [ ]:

audio_features_expanded = np.expand_dims(audio_features, axis=-1)
video_features_expanded = np.expand_dims(video_features, axis=-1)

(audio_train, audio_test, video_train, video_test, scores_train, scores_test) = train_test_split(
    audio_features_expanded, video_features_expanded, personality_scores, test_size=0.2, random_state=42
)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(((audio_train, video_train), scores_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(((audio_test, video_test), scores_test)).batch(32)

In [ ]:
encoder_a = create_encoder(input_shape=(audio_features.shape[1], 1), name="audio_encoder")
encoder_v = create_encoder(input_shape=(video_features.shape[1], 1), name="video_encoder")

In [39]:

dual_encoder = DualEncoder(encoder_a, encoder_v)
dual_encoder.compile(optimizer=keras.optimizers.Adam())


In [ ]:
num_epochs = 200
dual_encoder.fit(train_dataset, epochs=num_epochs)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 2.4921
Epoch 2/200
1/1 [==============================] - 0s 33ms/step - loss: 1.9583
Epoch 3/200
1/1 [==============================] - 0s 35ms/step - loss: 1.5853
Epoch 4/200
1/1 [==============================] - 0s 36ms/step - loss: 1.3426
Epoch 5/200
1/1 [==============================] - 0s 33ms/step - loss: 1.2106
Epoch 6/200
1/1 [==============================] - 0s 33ms/step - loss: 1.1411
Epoch 7/200
1/1 [==============================] - 0s 31ms/step - loss: 1.1098
Epoch 8/200
1/1 [==============================] - 0s 31ms/step - loss: 1.1089
Epoch 9/200
1/1 [==============================] - 0s 35ms/step - loss: 1.1181
Epoch 10/200
1/1 [==============================] - 0s 31ms/step - loss: 1.1258
Epoch 11/200
1/1 [==============================] - 0s 30ms/step - loss: 1.1250
Epoch 12/200
1/1 [==============================] - 0s 32ms/step - loss: 1.1158
Epoch 13/200
1/1 [==============================] -

In [ ]:
test_audio_embeddings, test_video_embeddings, predictions = dual_encoder.predict(test_dataset)

mse = mean_squared_error(scores_test, predictions)
r2 = r2_score(scores_test, predictions)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

1/1 [==============================] - 0s 160ms/step
Mean Squared Error: 0.027039427146574722
R-squared: -0.0390604869446296


# 

In [42]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras import layers, regularizers

In [43]:
video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')


In [ ]:
video_features = video_features_df.filter(regex='Feature_').values
audio_features = audio_features_df.drop(columns=['name']).iloc[:, :-5].values
personality_scores = video_features_df[['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']].values


In [ ]:
def create_enhanced_encoder(input_shape, name):
    input_layer = keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)  
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    return keras.Model(inputs=input_layer, outputs=x, name=name)


class DualEncoder(keras.Model):
    def __init__(self, encoder_a, encoder_v, **kwargs):
        super().__init__(**kwargs)
        self.encoder_a = encoder_a
        self.encoder_v = encoder_v
        self.regression_head = keras.layers.Dense(5, activation=None) 
        
    def call(self, inputs):
        audio_input, video_input = inputs
        audio_embedding = self.encoder_a(audio_input)
        video_embedding = self.encoder_v(video_input)
        combined_embedding = keras.layers.concatenate([audio_embedding, video_embedding], axis=-1)
        personality_scores_pred = self.regression_head(combined_embedding)
        return audio_embedding, video_embedding, personality_scores_pred

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            audio_embedding, video_embedding, personality_scores_pred = self(x, training=True)
            loss = self.compute_loss(y, personality_scores_pred, audio_embedding, video_embedding)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss}

    def compute_loss(self, true_scores, pred_scores, audio_embedding, video_embedding):
        mse_loss = tf.reduce_mean(keras.losses.mean_squared_error(true_scores, pred_scores))
        cosine_loss = tf.reduce_mean(1 - tf.keras.losses.cosine_similarity(audio_embedding, video_embedding))
        return mse_loss + cosine_loss


In [ ]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [ ]:
audio_features_expanded = np.expand_dims(audio_features, axis=-1)
video_features_expanded = np.expand_dims(video_features, axis=-1)
(audio_train, audio_test, video_train, video_test, scores_train, scores_test) = train_test_split(
    audio_features_expanded, video_features_expanded, personality_scores, test_size=0.2, random_state=42
)
train_dataset = tf.data.Dataset.from_tensor_slices(((audio_train, video_train), scores_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(((audio_test, video_test), scores_test)).batch(32)

encoder_a_enhanced = create_enhanced_encoder(input_shape=(audio_features.shape[1], 1), name="audio_encoder_enhanced")
encoder_v_enhanced = create_enhanced_encoder(input_shape=(video_features.shape[1], 1), name="video_encoder_enhanced")

dual_encoder_enhanced = DualEncoder(encoder_a_enhanced, encoder_v_enhanced)
dual_encoder_enhanced.compile(optimizer=keras.optimizers.Adam())

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
num_epochs = 2000  

dual_encoder_enhanced.fit(train_dataset, epochs=num_epochs, callbacks=[callback])

_, _, predictions_enhanced = dual_encoder_enhanced.predict(test_dataset)

mse_enhanced = mean_squared_error(scores_test, predictions_enhanced)
r2_enhanced = r2_score(scores_test, predictions_enhanced)

print(f"Enhanced Model - Mean Squared Error: {mse_enhanced}")
print(f"Enhanced Model - R-squared: {r2_enhanced}")

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - loss: 2.0741 - lr: 0.0010
Epoch 2/2000
1/1 [==============================] - 0s 108ms/step - loss: 1.4605 - lr: 0.0010
Epoch 3/2000
1/1 [==============================] - 0s 99ms/step - loss: 1.2983 - lr: 0.0010
Epoch 4/2000
1/1 [==============================] - 0s 15ms/step - loss: 1.3041 - lr: 0.0010
Epoch 5/2000
1/1 [==============================] - 0s 19ms/step - loss: 1.2285 - lr: 0.0010
Epoch 6/2000
1/1 [==============================] - 0s 22ms/step - loss: 1.1841 - lr: 9.0484e-04
Epoch 7/2000
1/1 [==============================] - 0s 18ms/step - loss: 1.1360 - lr: 8.1873e-04
Epoch 8/2000
1/1 [==============================] - 0s 16ms/step - loss: 1.1102 - lr: 7.4082e-04
Epoch 9/2000
1/1 [==============================] - 0s 17ms/step - loss: 1.1211 - lr: 6.7032e-04
Epoch 10/2000
1/1 [==============================] - 0s 18ms/step - loss: 1.1016 - lr: 6.0653e-04
Epoch 11/2000
1/1 [==============================]

In [ ]:
# improved code -------------------------------------------------------------------------------------------------------

# CL

In [49]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [50]:

video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')


In [ ]:

video_features = video_features_df.iloc[:, 1:-5].values 
audio_features = audio_features_df.iloc[:, 1:-5].values 

video_features = (video_features - np.mean(video_features, axis=0)) / np.std(video_features, axis=0)
audio_features = (audio_features - np.mean(audio_features, axis=0)) / np.std(audio_features, axis=0)
personality_scores = video_features_df[['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']].values

split_index = int(0.8 * len(video_features))  
train_video_features = video_features[:split_index]
test_video_features = video_features[split_index:]

train_audio_features = audio_features[:split_index]
test_audio_features = audio_features[split_index:]

train_personality_scores = personality_scores[:split_index]
test_personality_scores = personality_scores[split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((train_video_features, train_audio_features, train_personality_scores))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((test_video_features, test_audio_features, test_personality_scores))
test_dataset = test_dataset.batch(32)


In [ ]:
def prepare_features(video_features, audio_features, personality_scores):
    return (video_features, audio_features), personality_scores

train_dataset = tf.data.Dataset.from_tensor_slices((train_video_features, train_audio_features, train_personality_scores))
train_dataset = train_dataset.map(prepare_features)  
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((test_video_features, test_audio_features, test_personality_scores))
test_dataset = test_dataset.map(prepare_features) 
test_dataset = test_dataset.batch(32)



In [ ]:
class ContrastiveModel(keras.Model):
    def __init__(self, encoder_video, encoder_audio, predictor, **kwargs):
        super().__init__(**kwargs)
        self.encoder_video = encoder_video
        self.encoder_audio = encoder_audio
        self.predictor = predictor

    def call(self, inputs):
        video_input, audio_input = inputs
        video_embedding = self.encoder_video(video_input)
        audio_embedding = self.encoder_audio(audio_input)
        combined_embedding = tf.concat([video_embedding, audio_embedding], axis=1)
        personality_pred = self.predictor(combined_embedding)
        return video_embedding, audio_embedding, personality_pred

    def train_step(self, data):
  
        (video_input, audio_input), true_personality = data

        with tf.GradientTape() as tape:
            video_embedding, audio_embedding, personality_pred = self((video_input, audio_input), training=True)
           
            contrastive_loss = -tf.reduce_mean(tf.keras.losses.cosine_similarity(video_embedding, audio_embedding))
            
            regression_loss = tf.reduce_mean(keras.losses.MSE(true_personality, personality_pred))
           
            total_loss = contrastive_loss + regression_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"total_loss": total_loss, "contrastive_loss": contrastive_loss, "regression_loss": regression_loss}

encoder_video = keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
], name='encoder_video')

encoder_audio = keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
], name='encoder_audio')

predictor = keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation=None)  
], name='predictor')

model = ContrastiveModel(encoder_video, encoder_audio, predictor)
model.compile(optimizer=keras.optimizers.Adam(1e-4))

model.fit(train_dataset, epochs=10, validation_data=test_dataset)


Epoch 1/10
1/1 [==============================] - 1s 917ms/step - total_loss: 1.1420 - contrastive_loss: 0.2772 - regression_loss: 0.8647 - val_loss: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 48ms/step - total_loss: 0.9104 - contrastive_loss: 0.2707 - regression_loss: 0.6397 - val_loss: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 48ms/step - total_loss: 0.7347 - contrastive_loss: 0.2637 - regression_loss: 0.4709 - val_loss: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 48ms/step - total_loss: 0.6011 - contrastive_loss: 0.2565 - regression_loss: 0.3445 - val_loss: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 49ms/step - total_loss: 0.5013 - contrastive_loss: 0.2491 - regression_loss: 0.2523 - val_loss: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 46ms/step - total_loss: 0.4287 - contrastive_loss: 0.2410 - regression_loss: 0.1877 - val_loss: 0.0000e+00
Epoch 7/10
1/1 [=======================

In [58]:
for (inputs, targets) in train_dataset.take(1):
    print(f'Inputs structure: {type(inputs)}, {len(inputs)}')
    print(f'Targets shape: {targets.shape}')
    video_input, audio_input = inputs
    print(f'Video input shape: {video_input.shape}, Audio input shape: {audio_input.shape}')


Inputs structure: <class 'tuple'>, 2
Targets shape: (30, 5)
Video input shape: (30, 768), Audio input shape: (30, 1024)


In [59]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_embedding_alignment(model, test_data):
    video_embeddings, audio_embeddings = [], []
    for (video_input, audio_input), _ in test_data.unbatch().batch(1):
        video_emb, audio_emb, _ = model.predict((video_input, audio_input))
        video_embeddings.append(video_emb.flatten())
        audio_embeddings.append(audio_emb.flatten())
    
    video_embeddings = np.array(video_embeddings)
    audio_embeddings = np.array(audio_embeddings)
    similarity_scores = cosine_similarity(video_embeddings, audio_embeddings)
    mean_similarity = np.diag(similarity_scores).mean()
    print(f"Mean cosine similarity between corresponding video and audio embeddings: {mean_similarity:.4f}")

evaluate_embedding_alignment(model, test_dataset)


1/1 [==============================] - 0s 36ms/step
Mean cosine similarity between corresponding video and audio embeddings: 0.2598


In [60]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_personality_predictions(model, test_data):
    y_true, y_pred = [], []
    for (video_input, audio_input), personality_scores in test_data.unbatch().batch(1):
        _, _, predictions = model.predict((video_input, audio_input))
        y_true.append(personality_scores.numpy().flatten())
        y_pred.append(predictions.flatten())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    
    print(f"MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

evaluate_personality_predictions(model, test_dataset)


1/1 [==============================] - 0s 35ms/step
MSE: 0.4995, RMSE: 0.7068, MAE: 0.5684


# CL

In [ ]:
import os
import collections
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import pandas as pd


tf.get_logger().setLevel("ERROR")

In [ ]:

video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')

In [ ]:

video_features = video_features_df.iloc[:, 1:-5].values  
audio_features = audio_features_df.iloc[:, 1:-5].values  

video_features = (video_features - np.mean(video_features, axis=0)) / np.std(video_features, axis=0)
audio_features = (audio_features - np.mean(audio_features, axis=0)) / np.std(audio_features, axis=0)

personality_scores = video_features_df[['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']].values

# CL -------------------------- complete code

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')

video_features = video_features_df.iloc[:, 1:-5].values  
audio_features = audio_features_df.iloc[:, 1:-5].values 

video_features = (video_features - np.mean(video_features, axis=0)) / np.std(video_features, axis=0)
audio_features = (audio_features - np.mean(audio_features, axis=0)) / np.std(audio_features, axis=0)


split_index = int(0.8 * len(video_features)) 
train_video_features = video_features[:split_index]
test_video_features = video_features[split_index:]

train_audio_features = audio_features[:split_index]
test_audio_features = audio_features[split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices(({"video": train_video_features, "audio": train_audio_features}))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices(({"video": test_video_features, "audio": test_audio_features}))
test_dataset = test_dataset.batch(32)

def create_video_encoder():
    input_video = tf.keras.Input(shape=[video_features.shape[1]]) 
    x1 = tf.keras.layers.Reshape((video_features.shape[1], 1))(input_video)  
    x1 = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same', name="cnn1")(x1)
    x1 = tf.keras.layers.MaxPooling1D(name="maxpool1")(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Dense(120, activation=None)(x1)
    return tf.keras.Model(inputs=input_video, outputs=x1, name="video_encoder")

def create_audio_encoder():
    input_audio = tf.keras.Input(shape=[audio_features.shape[1]])  
    x2 = tf.keras.layers.Reshape((audio_features.shape[1], 1))(input_audio)  
    x2 = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(x2)
    x2 = tf.keras.layers.MaxPooling1D()(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Dense(120, activation=None)(x2)
    return tf.keras.Model(inputs=input_audio, outputs=x2, name="audio_encoder")

class TripleEncoder(tf.keras.Model):
    def __init__(self, video_encoder, audio_encoder, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.video_encoder = video_encoder
        self.audio_encoder = audio_encoder
        self.temperature = temperature
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        video_embeddings = self.video_encoder(features["video"], training=training)
        audio_embeddings = self.audio_encoder(features["audio"], training=training)
        return video_embeddings, audio_embeddings

    def compute_loss(self, embeddings1, embeddings2):
        logits = tf.matmul(embeddings1, embeddings2, transpose_b=True) / self.temperature
        similarity_matrix = tf.matmul(embeddings2, embeddings2, transpose_b=True)
        targets = tf.keras.activations.softmax(similarity_matrix / (2 * self.temperature))
        loss = tf.keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        return loss

    def train_step(self, features):
        with tf.GradientTape() as tape:
            video_embeddings, audio_embeddings = self(features, training=True)
            video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
            total_loss = video_audio_loss
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, features):
        video_embeddings, audio_embeddings = self(features, training=False)
        video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
        total_loss = video_audio_loss
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

video_encoder = create_video_encoder()
audio_encoder = create_audio_encoder()

triple_encoder = TripleEncoder(video_encoder, audio_encoder, temperature=1.0)

triple_encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

num_epochs = 50
history = triple_encoder.fit(train_dataset, epochs=num_epochs)

evaluation_result = triple_encoder.evaluate(test_dataset)
print("Evaluation Loss:", evaluation_result)

Epoch 1/50
1/1 [==============================] - 1s 780ms/step - loss: 3.9933
Epoch 2/50
1/1 [==============================] - 0s 50ms/step - loss: 3.7940
Epoch 3/50
1/1 [==============================] - 0s 50ms/step - loss: 1.8516
Epoch 4/50
1/1 [==============================] - 0s 61ms/step - loss: 1.6175
Epoch 5/50
1/1 [==============================] - 0s 52ms/step - loss: 0.8527
Epoch 6/50
1/1 [==============================] - 0s 50ms/step - loss: 0.7359
Epoch 7/50
1/1 [==============================] - 0s 51ms/step - loss: 0.8303
Epoch 8/50
1/1 [==============================] - 0s 60ms/step - loss: 0.5387
Epoch 9/50
1/1 [==============================] - 0s 26ms/step - loss: 0.6356
Epoch 10/50
1/1 [==============================] - 0s 27ms/step - loss: 0.4450
Epoch 11/50
1/1 [==============================] - 0s 27ms/step - loss: 0.4071
Epoch 12/50
1/1 [==============================] - 0s 26ms/step - loss: 0.4159
Epoch 13/50
1/1 [==============================] - 0s 28ms/s

In [3]:
# saving the model

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')

video_features = video_features_df.iloc[:, 1:-5].values  
audio_features = audio_features_df.iloc[:, 1:-5].values 

video_features = (video_features - np.mean(video_features, axis=0)) / np.std(video_features, axis=0)
audio_features = (audio_features - np.mean(audio_features, axis=0)) / np.std(audio_features, axis=0)

dataset = tf.data.Dataset.from_tensor_slices(({"video": video_features, "audio": audio_features}))
dataset = dataset.shuffle(buffer_size=1024).batch(32)

In [ ]:
# TripleEncoder MEANS DUAL ENCODER

In [5]:
def create_video_encoder():
    input_video = tf.keras.Input(shape=[video_features.shape[1]])  # Assuming shape of video features
    x1 = tf.keras.layers.Reshape((video_features.shape[1], 1))(input_video)  # Adding a channel dimension
    x1 = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same', name="cnn1")(x1)
    x1 = tf.keras.layers.MaxPooling1D(name="maxpool1")(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Dense(120, activation=None)(x1)
    return tf.keras.Model(inputs=input_video, outputs=x1, name="video_encoder")

def create_audio_encoder():
    input_audio = tf.keras.Input(shape=[audio_features.shape[1]])  # Assuming shape of audio features
    x2 = tf.keras.layers.Reshape((audio_features.shape[1], 1))(input_audio)  # Adding a channel dimension
    x2 = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(x2)
    x2 = tf.keras.layers.MaxPooling1D()(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Dense(120, activation=None)(x2)
    return tf.keras.Model(inputs=input_audio, outputs=x2, name="audio_encoder")

class TripleEncoder(tf.keras.Model):
    def __init__(self, video_encoder, audio_encoder, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.video_encoder = video_encoder
        self.audio_encoder = audio_encoder
        self.temperature = temperature
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        video_embeddings = self.video_encoder(features["video"], training=training)
        audio_embeddings = self.audio_encoder(features["audio"], training=training)
        return video_embeddings, audio_embeddings

    def compute_loss(self, embeddings1, embeddings2):
        logits = tf.matmul(embeddings1, embeddings2, transpose_b=True) / self.temperature
        similarity_matrix = tf.matmul(embeddings2, embeddings2, transpose_b=True)
        targets = tf.keras.activations.softmax(similarity_matrix / (2 * self.temperature))
        loss = tf.keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        return loss

    def train_step(self, features):
        with tf.GradientTape() as tape:
            video_embeddings, audio_embeddings = self(features, training=True)
            video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
            total_loss = video_audio_loss
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, features):
        video_embeddings, audio_embeddings = self(features, training=False)
        video_audio_loss = self.compute_loss(video_embeddings, audio_embeddings)
        total_loss = video_audio_loss
        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}


In [ ]:
video_encoder = create_video_encoder()
audio_encoder = create_audio_encoder()


In [ ]:

triple_encoder = TripleEncoder(video_encoder, audio_encoder, temperature=1.0)

triple_encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))


In [ ]:
num_epochs = 50
history = triple_encoder.fit(dataset, epochs=num_epochs)

triple_encoder.save("C:/Users/giris/Downloads/RESEARCH/Personality/MODEL/triple_encoder_model")


Epoch 1/50
2/2 [==============================] - 1s 38ms/step - loss: 3.1695
Epoch 2/50
2/2 [==============================] - 0s 10ms/step - loss: 6.1977
Epoch 3/50
2/2 [==============================] - 0s 8ms/step - loss: 1.1963
Epoch 4/50
2/2 [==============================] - 0s 9ms/step - loss: 4.9367
Epoch 5/50
2/2 [==============================] - 0s 8ms/step - loss: 0.1251
Epoch 6/50
2/2 [==============================] - 0s 8ms/step - loss: 2.4799e-04
Epoch 7/50
2/2 [==============================] - 0s 6ms/step - loss: 0.0247
Epoch 8/50
2/2 [==============================] - 0s 8ms/step - loss: 2.1015
Epoch 9/50
2/2 [==============================] - 0s 8ms/step - loss: 0.4138
Epoch 10/50
2/2 [==============================] - 0s 7ms/step - loss: 0.1344
Epoch 11/50
2/2 [==============================] - 0s 8ms/step - loss: 1.4111e-04
Epoch 12/50
2/2 [==============================] - 0s 7ms/step - loss: 6.3470e-04
Epoch 13/50
2/2 [==============================] - 0s 8ms/s

INFO:tensorflow:Assets written to: C:/Users/giris/Downloads/RESEARCH/Personality/MODEL/triple_encoder_model\assets


INFO:tensorflow:Assets written to: C:/Users/giris/Downloads/RESEARCH/Personality/MODEL/triple_encoder_model\assets


In [ ]:
loaded_model = tf.keras.models.load_model("C:/Users/giris/Downloads/RESEARCH/Personality/MODEL/triple_encoder_model")

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
video_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Video_Interview_features.csv')
audio_features_df = pd.read_csv('C:/Users/giris/Downloads/RESEARCH/Personality/personality_data/Extracted_Audio_features.csv')

video_features = video_features_df.iloc[:, 1:-5].values  
audio_features = audio_features_df.iloc[:, 1:-5].values 

new_data = {"video": video_features, "audio": audio_features}

new_video_embeddings, new_audio_embeddings = loaded_model(new_data)

In [ ]:
print("Shape of new video embeddings:", new_video_embeddings.shape)
print("Shape of new audio embeddings:", new_audio_embeddings.shape)


Shape of new video embeddings: (38, 120)
Shape of new audio embeddings: (38, 120)
